In [3]:
import torch
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

'cuda'

In [5]:
with open('data/tiny_shakespeare.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [6]:
len(text)

1115394

### Vocab

In [7]:
unique_symbols = sorted(list(set(text)))
vocab_size = len(unique_symbols)

print(''.join(unique_symbols))
print(f'{vocab_size=}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocab_size=65


### Encoder/decoder

In [8]:
stoi = {s: i for i, s in enumerate(unique_symbols)}
itos = {i: s for s, i in stoi.items()}

encode = lambda s: [stoi[ch] for ch in s]  # symbols to tokens
decode = lambda l: ''.join([itos[i] for i in l])  # tokens to symbols

s = 'hii there!'
print(encode(s))
print(decode(encode(s)))

[46, 47, 47, 1, 58, 46, 43, 56, 43, 2]
hii there!


### Train/val split

In [9]:
import torch

data = torch.tensor(encode(text), dtype = torch.long)
data.shape

torch.Size([1115394])

In [10]:
n = int(0.9 * len(data))

train_data = data[:n]
val_data = data[n:]

print(train_data.shape, val_data.shape)

torch.Size([1003854]) torch.Size([111540])


### Build a dataset

In [11]:
block_size = 8
batch_size = 4

In [12]:
## example of how data is going to look like
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for i in range(block_size):
    x_i = x[:i+1]
    y_i = y[i]
    print(f'when input is {x_i} the target is {y_i}')

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [13]:
torch.manual_seed(1337)

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x_batch = torch.stack([data[i:i+block_size] for i in ix])
    y_batch = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x_batch, y_batch = x_batch.to(device), y_batch.to(device)

    return x_batch, y_batch

In [14]:
x_batch, y_batch = get_batch('train')
print(x_batch.shape, y_batch.shape)

torch.Size([4, 8]) torch.Size([4, 8])


In [15]:
# check that our batch was formed correctly
for batch_i in range(batch_size):
    for i in range(block_size):
        x_i = x_batch[batch_i, :i+1]
        y_i = y_batch[batch_i, i]
        print(f'when input is {x_i} the target is {y_i}')

when input is tensor([24], device='cuda:0') the target is 43
when input is tensor([24, 43], device='cuda:0') the target is 58
when input is tensor([24, 43, 58], device='cuda:0') the target is 5
when input is tensor([24, 43, 58,  5], device='cuda:0') the target is 57
when input is tensor([24, 43, 58,  5, 57], device='cuda:0') the target is 1
when input is tensor([24, 43, 58,  5, 57,  1], device='cuda:0') the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46], device='cuda:0') the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43], device='cuda:0') the target is 39
when input is tensor([44], device='cuda:0') the target is 53
when input is tensor([44, 53], device='cuda:0') the target is 56
when input is tensor([44, 53, 56], device='cuda:0') the target is 1
when input is tensor([44, 53, 56,  1], device='cuda:0') the target is 58
when input is tensor([44, 53, 56,  1, 58], device='cuda:0') the target is 46
when input is tensor([44, 53, 56,  1, 58, 46], device='cud

## Bigram model

In [16]:
import torch.nn as nn
from torch.nn import functional as F

In [17]:
class BigramModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None):
        # idx in our case is [batch, block_szie]
        # so logits is gonna be [batch, block_szie, vector_dim], where vector_dim  = vocab_size
        logits = self.token_embedding_table(idx) # (B, T, C), where B-batch, T-time, C-channel

        if targets == None:
            loss = None
        else:
            # logits (B, T, C)
            # targets (B, T)
            # but F.cross_entropy expects input of shape (batch, C, <all other dims>) ---> error
            B, T, C = logits.shape
            loss = F.cross_entropy(logits.view(B*T, C), targets.view(-1))
            
        return logits, loss


    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(idx)
            logits = logits[:, -1, :] # take only last token to predict the next one
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples= 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

### Before training

In [18]:
m = BigramModel(vocab_size)
m.to(device)
logits, loss = m(x_batch, y_batch)
print(loss)

tensor(5.0364, device='cuda:0', grad_fn=<NllLossBackward0>)


In [19]:
m.token_embedding_table.weight 

Parameter containing:
tensor([[ 0.6258,  0.0255,  0.9545,  ...,  0.0688,  1.3327, -0.4970],
        [ 0.4658, -0.2573, -1.0673,  ...,  1.2439,  1.3471,  1.6910],
        [-0.1244, -1.6824,  1.1346,  ...,  1.0785, -0.6150, -0.4589],
        ...,
        [ 0.1373,  0.2902, -0.1721,  ...,  2.9050,  1.3809,  0.5141],
        [ 0.0195,  0.3881,  0.5838,  ...,  0.1362, -0.2022, -1.8831],
        [-0.5178, -0.0930,  0.7448,  ...,  0.0883,  2.3935, -0.7376]],
       device='cuda:0', requires_grad=True)

In [20]:
encode('\n')

[0]

In [22]:
start_token = torch.zeros((1, 1), dtype= torch.int, device = device)
gen_text = m.generate(start_token, 100)
print(decode(gen_text[0].tolist()))


yq$;tfBfROkNdcuwdZZTkOMl;,ertK
w:!PLCkMBbeA$3:XaSGJO-3p&M-c?KL3auhpFYVXJFhNNNuhq$OMxv.tbVFYdXlrFZaAe


### Training

In [23]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

In [24]:
batch_size = 32
for step in range(10_000):
    x_batch, y_batch = get_batch('train')
    logits, loss = m(x_batch, y_batch)

    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())    

2.4487955570220947


In [25]:
start_token = torch.zeros((1, 1), dtype= torch.int, device = device)
gen_text = m.generate(start_token, 100)
print(decode(gen_text[0].tolist()))

# generated text looks better then before training but it is still not good 


Wawice my.

HDEdarom oroup
Yowhthetof isth ble mil; dill, ath iree sengmin lat Heriliovets, and Win 
